# Clippy, the research assistant! 
#### Now on AWS!

### Importing packages and setting client details

In [1]:
# Importing packages
import json
import os
import re
import pandas as pd
from anthropic import AnthropicBedrock
import time
from botocore.exceptions import ClientError
from ratelimit import limits, sleep_and_retry
import anthropic

# Set environment variables
os.environ['AWS_ACCESS_KEY']='AWS_ACCESS_KEY'
os.environ['AWS_SECRET_KEY']='AWS_SECRET_KEY'
os.environ['REGION_NAME']='REGION_NAME'

AWS_ACCESS_KEY=os.getenv('AWS_ACCESS_KEY')
AWS_SECRET_KEY=os.getenv('AWS_SECRET_KEY')
REGION_NAME=os.getenv('REGION_NAME')

### Unleashing Clippy to do his thing!

In [26]:
# Changing directory to papers folder for mac
os.chdir("FILE_PATH")
df = pd.read_csv('CSV_FILE_NAME')

# Filling in blanks with NA text
clean_df=df.fillna(value=('Missing'))

# Create an empty list to append
results=[]

# Initiating client
client = AnthropicBedrock(
    aws_access_key=AWS_ACCESS_KEY,
    aws_secret_key=AWS_SECRET_KEY,
    aws_region=REGION_NAME,
)

# Abstract to feed in
for index, row in clean_df.iterrows():
  time.sleep(30) # setting a buffer to 30 seconds to reduce likelihood of a choke
  abstract=row['Abstract']
  response = client.messages.create(
          model="anthropic.claude-3-haiku-20240307-v1:0",
          max_tokens=1000, 
          temperature=0.1,
          system='''You are Clippy, an AI research assistant. \
                    Evaluate provided research paper abstracts \
                    for relevance to a literature review on Large \
                    Language Models in patient self-care. Respond \
                        in JSON format with these fields:
                    {'relevance_yes_no': yes/no,
                     'relevance': One-sentence explanation of relevance,
                     'purpose_of_study': Brief description of research purpose}\
                    Be concise. If uncertain: 'relevance_yes_no': "Review full text", others: "Insufficient information"''',
          messages=[{'role': 'user', 'content': abstract}]
  ) 
  results.append(pd.json_normalize(json.loads(response.content[0].text)))
# Appending to blank results list
data_for_append=pd.concat(results)

# Stick the columns together using concat
lit_review=pd.concat([clean_df, data_for_append.reset_index(drop=True)], axis=1)

# Output to csv file
lit_review.to_excel('FILE_NAME', index=False)